# <center> <font color = 'rebeccapurple'>Stochastic Watershed Model: Generating Stochastic Streamflows for the Shasta River Under 5 Degrees of Warming</font> </center>

This Jupyter Notebook contains code for a stochastic watershed model (SWM) that can be used to develop an ensemble of stochastic streamflows for a single site. This code was used to produce the results found in **Shabestanipour et al. (2024)** for the Shasta River basin. With modifications to the input data and the specific ARMA model used (more details below), this code could be applied to other non-regulated river basins. 

This notebook is split into two parts: 1. SWM (generating stochastic streamflows) and 2. Diagnostics (assessing stochastic model performance).

***

**Computational Requirements**: We used Jupyter version 4.7.1 and Python version 3.8.8 to run the below code. Lower versions may result in error messages.

The code is somewhat computationally-intesnive. We found that allocating 32GB of memory allowed the code to run in full without crashing the kernel. 

The code blocks that will take the longest are generating the stochastic residuals in Part 1 and generating the figure inputs in Part 2 (depending on how many stochastic realizations you do). These should each take approx. 10 min maximum. Most other code blocks run very quickly. The code to generate AIC/BIC for various ARIMA models to determine which to use may also take some time (by default this code block is commented out).

***

# <center> <font color = 'red'> Part 1: SWM </font> </center>

## <font color='green'> Import Statements </font>

Here we import SWM functions (developed by Ghazal Shabestanipour) as well as existing external Python packages used in the notebook. Be sure that the file **Func_Lib.py** is contained in the same directory as this Jupyter Notebook.
For existing Python packages, if you do not have a particular package installed with your version of Python, you can install it using pip or conda (if using anaconda). Instructions for doing so can be found here:<br>
https://packaging.python.org/en/latest/tutorials/installing-packages/ (pip)<br>
https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/installing-with-conda.html (conda)

In [1]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

## <font color='green'> Load Input Data </font>

The only input data required is a csv containing daily streamflow data (observed and modeled). The columns of the csv are $date$, observed streamflow $Qgage$, deterministic model of streamflow $Qmodel$, and the difference between Qgage and Qmodel for each date $diff$.

For the Squannacook, we have values from 10/01/1987 - 09/30/2018.

In [2]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [3]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-15.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.007832,0.412647,82.444383
1,1,1987-10-02,0.007706,0.412069,82.328811
2,2,1987-10-03,0.007573,0.411455,82.206250
3,3,1987-10-04,0.007445,0.410867,82.088700
4,4,1987-10-05,0.007309,0.410235,81.962435
...,...,...,...,...,...
11318,11318,2018-09-26,0.010889,0.378551,75.632213
11319,11319,2018-09-27,0.010649,0.377398,75.401685
11320,11320,2018-09-28,0.010401,0.376197,75.161795
11321,11321,2018-09-29,0.011540,0.381652,76.251673


## <font color='green'> Fit ARMA Model to Data </font>

Here we fit an AR4 model to our "lambdas" to get the model residuals and parameters.

In [4]:
p=4 #order of AR (autocorrelation with lag 4)
d=0 #order of I (no trend)
q=0 #order of MA (no moving average component)

mod = ARIMA(data['lambda'], order=(p,d,q)) #set up AR4 model using lambdas as input
res = mod.fit() #fit model

In [5]:
res.params #display the model parameters

const     0.023440
ar.L1     0.165057
ar.L2     0.187447
ar.L3     0.147261
ar.L4     0.137038
sigma2    0.067909
dtype: float64

Since we fit an AR4 model for this data, we have five parameters: a coefficient for each AR component (L1, L2, L3, and L4) and a constant term.

In [6]:
#create a column in data for the model residuals associated with each time step
data['Ar_res']=res.resid
data.head(5)

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037


## <font color = 'green'> Generate Random AR Parameter Sets </font>

We generate $m$ random AR parameter sets (called "Beta") to consider the uncertainty in the AR model parameters.

We use the Numpy function **random.multivariate_normal** to generate the parameter sets. This function samples the parameters from a multivariate normal distribution. The inputs to the function are the mean **(res.params)**, the covariance **(res.cov_params)**, and the number of parameter sets to generate. 

In [7]:
# m is the number of random AR parameter sets, Beta, generated for considering AR model uncertainty.
m = 100

# older version of ARIMA function
#Beta=np.random.multivariate_normal(np.array(res.params), np.array(res.cov_params()), m)

# updated Beta for using newer version of statsmodels ARIMA code
# Beta=np.random.multivariate_normal(np.array(res.params[:-1]), np.array(res.cov_params())[:-1,:-1], m)

# for reproducible Betas, use scipy.stats multivariate_normal function instead, setting random seed
Beta = stats.multivariate_normal(np.array(res.params[:-1]),np.array(res.cov_params())[:-1,:-1], seed=16)
Beta = Beta.rvs(m)

## <font color = 'green'> Generate Stochastic Residuals Using Simple Bootstrap </font>

There are 3 steps to generating the stochastic streamflow realizations:<br>

1. Generate stochastic residuals (use either simple bootstrap, conditional bootstrap, or k-Nearest Neighbors bootstrap)
2. Generate stochastic log errors by combining AR model uncertainty + stochastic residuals in AR4 model
3. Generate stochastic streamflow by transforming from log-ratios back to streamflows, including bias correction factor<br>

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 1: Generate Stochastic Residuals <green>

In [8]:
#now find the nearest Qmodel and residual associated with Qmodel for each flow value in Qfut
df_fut['Qmodel_nearest'] = 0 #initialize empty column for nearest Qmodel
df_fut['residual_nearest'] = 0 #initialize empty column for nearest Qmodel residual

for i in range(len(df_fut)): #loop through each row of df_fut
    #for each flow, calculate absolute difference of that flow and all flows in Qmodel
    #take absolute value and then find the index of the minimum absolute value
    #populate Qmodel_nearest column with data['Qmodel'] at this index
    #and residual_nearest column with data['Ar_res'] at this index
    df_fut['Qmodel_nearest'].iloc[i] = data['Qmodel'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    df_fut['residual_nearest'].iloc[i] = data['Ar_res'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    
#now use KNN bootstrapping as before, with Epsilon defined as the Qmodel_nearest and residual_nearest columns from df_fut
#basically exactly what we had before, with some data "repeats" to correspond to the new data points
#and in the correct order to match Qfut

n = 100 # n is the number of stochastic streamflow realizations generated for each Beta set
Epsilon = df_fut.filter(['residual_nearest','Qmodel_nearest'])
Epsilon = np.array(Epsilon) # turn dataframe into numpy array

E = [] #initialize empty list to store errors
for t in range(0,len(df_fut)): # loop through all time periods
    E.append(Epsilon - [0,Epsilon[t,1]])  # first column residual, second column diff between flow and all other flows

k = 750

Er = np.zeros((len(df_fut),k)) #initialize empty 2d array with dimensions time periods x k (# nearest neighbors)
for t in range(0,len(df_fut)): #loop through all time periods
    E[t][:,1] = np.abs(E[t][:,1]) #get absolute values of errors
    x = E[t][np.argsort(E[t][:, 1])] #sort errors by magnitude
    Er[t,:] = x[0:k,0]
    

R = np.zeros((len(df_fut),n*m)) #initialize empty 2d array with dimensions time periods x # realizations
for i in range(0,n*m): #loop through all realizations
    I = np.random.randint(low=0, high=k, size=len(df_fut)) #randomly identify indices
    R[:,i] = Er[tuple(np.arange(0,len(df_fut),1)),I] #sample from Er at random indices I
    
E = R #set E to equal R

/cluster/tufts/hpc/tools/anaconda/202105/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


One needs to experiment with different k values to find the appropriate k for each basin by going trough all the next steps of the SWM and diagnostic figures.  

### <font color='green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 2: Generate Stochastic Log Errors </font>

In [9]:
# l is a matrix of [len(data),m*n] representing stochastic log-ratio errors         
L = np.zeros((len(df_fut),m*n)) #initialize empty array to populate below

# for the first five time periods, we set values to be the deterministic model lambdas across all realizations
# this is because we do not have enough time periods in the past to populate the AR4 model stochastically
for i in range(0,m*n):
    L[0:5,i] = data['lambda'].iloc[0:5] 

for i in range(0,m):    
    B = Beta[i,:] #let B = random parameter set for realization i
    for t in range(5,len(df_fut)):
        # L(t) = B0 + B1*L[t-1] + B2*L[t-2] + B3*L[t-3] + B4*L[t-4] + E[t]
        L[t, i*n:((i+1)*n)] = B[0] + \
                              B[1] * L[t-1, i*n:((i+1)*n)] + \
                              B[2] * L[t-2, i*n:((i+1)*n)] + \
                              B[3] * L[t-3, i*n:((i+1)*n)] + \
                              B[4] * L[t-4, i*n:((i+1)*n)] + \
                              E[t, i*n:((i+1)*n)]

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 3: Generate Stochastic Streamflows </font>

In [10]:
# Q is a matrix of stochastically simulated streamflows of size [m*n, len(data)]
# Just like lambda = log(Qmodel/Qgage), we now set Q = Qmodel/e^L where
# L is stochastic lambdas and Q is stochastic streamflow
Q = np.array(df_fut['Qfut']) / np.exp(L.T)

# BCF is transformation bias correction factor 
# this address the bias introduced by moving to log space
BCF = 1 / np.exp(-np.mean(data['lambda']) + np.var(data['lambda'])/2)

Q = Q*BCF

In [11]:
print(BCF)

0.9807641078478005


In [12]:
display(pd.DataFrame(Q))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.232358,10.96819,14.638944,15.264148,23.854913,27.190406,34.507014,29.782448,45.896559,48.282608,...,59.204429,57.152597,65.727812,60.997380,59.741933,59.978923,60.281948,59.482677,88.784350,91.844906
1,13.232358,10.96819,14.638944,15.264148,23.854913,33.072331,56.469877,42.491254,57.878284,82.026500,...,56.793930,56.545006,106.782392,75.989648,77.077209,69.109480,71.451962,67.101867,84.232657,89.912226
2,13.232358,10.96819,14.638944,15.264148,23.854913,24.874687,34.887349,61.095291,55.956692,62.674503,...,88.502163,70.203165,72.841657,66.618488,67.850596,65.008225,64.240237,66.290486,62.682336,95.822454
3,13.232358,10.96819,14.638944,15.264148,23.854913,25.824294,38.173495,47.316583,38.802204,48.775155,...,70.087635,69.250899,67.916606,62.605630,67.319072,62.261996,61.330080,61.798179,67.659676,98.140016
4,13.232358,10.96819,14.638944,15.264148,23.854913,30.265532,30.485594,37.554157,51.381139,43.321416,...,49.904861,51.173550,55.363585,30.942191,74.059695,52.586887,54.550423,51.061579,57.473202,101.559380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.232358,10.96819,14.638944,15.264148,23.854913,30.072097,33.066594,34.021789,47.640009,52.059199,...,67.165362,75.502008,94.479973,70.391883,67.453715,67.547100,67.059942,34.583584,65.297514,75.453129
9996,13.232358,10.96819,14.638944,15.264148,23.854913,14.972736,32.149889,33.276340,36.904565,45.037064,...,66.076283,82.574305,68.434911,66.770923,64.699212,66.050739,63.023629,62.195583,62.458296,86.066877
9997,13.232358,10.96819,14.638944,15.264148,23.854913,31.496325,48.010568,40.352993,59.218164,25.768368,...,73.456157,59.684890,69.600583,66.472447,63.648267,61.228742,62.138407,60.984183,60.544480,90.487050
9998,13.232358,10.96819,14.638944,15.264148,23.854913,32.424642,35.560290,42.427201,51.730404,42.425128,...,71.221026,70.540299,65.960311,69.493103,97.125150,68.346192,68.498170,63.834259,67.578539,80.354536


In [13]:
display(pd.DataFrame(L.T))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,1.810035,1.996299,1.706123,1.66287,1.214848,1.082520,0.842803,0.988935,0.555184,0.502998,...,0.240416,0.272227,0.129337,0.201483,0.219508,0.212466,0.204374,0.214535,-0.171594,0.025303
1,1.810035,1.996299,1.706123,1.66287,1.214848,0.886687,0.350258,0.633556,0.323232,-0.026973,...,0.281983,0.282915,-0.355934,-0.018283,-0.035265,0.070767,0.034381,0.094008,-0.118967,0.046571
2,1.810035,1.996299,1.706123,1.66287,1.214848,1.171534,0.831841,0.270420,0.356996,0.242114,...,-0.161615,0.066558,0.026571,0.113332,0.092234,0.131945,0.140777,0.106173,0.176536,-0.017092
3,1.810035,1.996299,1.706123,1.66287,1.214848,1.134069,0.741824,0.525994,0.723097,0.492848,...,0.071666,0.080215,0.096578,0.175459,0.100099,0.175108,0.187136,0.176346,0.100126,-0.040991
4,1.810035,1.996299,1.706123,1.66287,1.214848,0.975375,0.966711,0.757070,0.442303,0.611422,...,0.411294,0.382729,0.300937,0.880194,0.004671,0.343992,0.304281,0.367187,0.263297,-0.075239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.810035,1.996299,1.706123,1.66287,1.214848,0.981787,0.885442,0.855853,0.517901,0.427688,...,0.114255,-0.006208,-0.233529,0.058236,0.098101,0.093634,0.097819,0.756841,0.135662,0.221893
9996,1.810035,1.996299,1.706123,1.66287,1.214848,1.679153,0.913557,0.878008,0.773239,0.572583,...,0.130603,-0.095747,0.088976,0.111047,0.139793,0.116036,0.159897,0.169936,0.180117,0.090280
9997,1.810035,1.996299,1.706123,1.66287,1.214848,0.935514,0.512545,0.685189,0.300346,1.130921,...,0.024724,0.228873,0.072086,0.115527,0.156170,0.191842,0.174042,0.189605,0.211238,0.040198
9998,1.810035,1.996299,1.706123,1.66287,1.214848,0.906466,0.812736,0.635065,0.435528,0.632328,...,0.055624,0.061767,0.125806,0.071087,-0.266458,0.081873,0.076599,0.143930,0.101326,0.158956


In [14]:
data

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037
...,...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934,-0.022115
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129,-0.024599
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849,0.261246
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763,-0.024250


## <font color = 'green'> Save Outputs </font>

Outputs saved as **.npy** files. The outputs are:

1. E.npy: stochastic residuals
2. L.npy: stochastic log ratios
3. Q.npy: stochastic streamflows

In [15]:
# save the results as .npy files
np.save('Q_5degree.npy',Q)
np.save('StochasticE_5degree.npy',E)
np.save('L_5degree.npy',L)

***

# <center> <font color = 'red'> Part 2: Diagnostic Figures </font> </center>

Here, we include code to generate T-year flood flows for the specified warming scenario.

## <font color = 'green'> Reloading in the data </font>

In [16]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

In [17]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [18]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-15.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.007832,0.412647,82.444383
1,1,1987-10-02,0.007706,0.412069,82.328811
2,2,1987-10-03,0.007573,0.411455,82.206250
3,3,1987-10-04,0.007445,0.410867,82.088700
4,4,1987-10-05,0.007309,0.410235,81.962435
...,...,...,...,...,...
11318,11318,2018-09-26,0.010889,0.378551,75.632213
11319,11319,2018-09-27,0.010649,0.377398,75.401685
11320,11320,2018-09-28,0.010401,0.376197,75.161795
11321,11321,2018-09-29,0.011540,0.381652,76.251673


In [19]:
Q = np.load('Q_5degree.npy')
m=100
n=100

In [20]:
pd.DataFrame(Q)

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.232358,10.96819,14.638944,15.264148,23.854913,27.190406,34.507014,29.782448,45.896559,48.282608,...,59.204429,57.152597,65.727812,60.997380,59.741933,59.978923,60.281948,59.482677,88.784350,91.844906
1,13.232358,10.96819,14.638944,15.264148,23.854913,33.072331,56.469877,42.491254,57.878284,82.026500,...,56.793930,56.545006,106.782392,75.989648,77.077209,69.109480,71.451962,67.101867,84.232657,89.912226
2,13.232358,10.96819,14.638944,15.264148,23.854913,24.874687,34.887349,61.095291,55.956692,62.674503,...,88.502163,70.203165,72.841657,66.618488,67.850596,65.008225,64.240237,66.290486,62.682336,95.822454
3,13.232358,10.96819,14.638944,15.264148,23.854913,25.824294,38.173495,47.316583,38.802204,48.775155,...,70.087635,69.250899,67.916606,62.605630,67.319072,62.261996,61.330080,61.798179,67.659676,98.140016
4,13.232358,10.96819,14.638944,15.264148,23.854913,30.265532,30.485594,37.554157,51.381139,43.321416,...,49.904861,51.173550,55.363585,30.942191,74.059695,52.586887,54.550423,51.061579,57.473202,101.559380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.232358,10.96819,14.638944,15.264148,23.854913,30.072097,33.066594,34.021789,47.640009,52.059199,...,67.165362,75.502008,94.479973,70.391883,67.453715,67.547100,67.059942,34.583584,65.297514,75.453129
9996,13.232358,10.96819,14.638944,15.264148,23.854913,14.972736,32.149889,33.276340,36.904565,45.037064,...,66.076283,82.574305,68.434911,66.770923,64.699212,66.050739,63.023629,62.195583,62.458296,86.066877
9997,13.232358,10.96819,14.638944,15.264148,23.854913,31.496325,48.010568,40.352993,59.218164,25.768368,...,73.456157,59.684890,69.600583,66.472447,63.648267,61.228742,62.138407,60.984183,60.544480,90.487050
9998,13.232358,10.96819,14.638944,15.264148,23.854913,32.424642,35.560290,42.427201,51.730404,42.425128,...,71.221026,70.540299,65.960311,69.493103,97.125150,68.346192,68.498170,63.834259,67.578539,80.354536


## <font color = 'green'> Generating Figure Inputs </font>

In [21]:
def annualmax(Q,data,n):
    annual_max=np.array(pd.DataFrame(Q.T,index=data['date']).resample('Y').max())
    annualmaxsort = np.zeros(annual_max.shape)
    for i in range(0,n):
        annualmaxsort[:,i]=np.sort(annual_max[:,i])[::-1]
    
    annualmaxsort = pd.DataFrame(annualmaxsort)
    #annualmaxsort['rank']=(annualmaxsort.index) + 1
    #annualmaxsort['Exeedance']=annualmaxsort['rank']/(len(annualmaxsort)+1)
    #annualmaxsort['non_Exceedance']=1-annualmaxsort['Exeedance']
    #Data=data.set_index('date')
    #annualmaxobs=Data['Qgage'].resample('Y').max()
    #annualmaxmodel=Data['Qmodel'].resample('Y').max()
    #annualmaxsort['obs']=np.sort(annualmaxobs)[::-1]
    #annualmaxsort['model']=np.sort(annualmaxmodel)[::-1]
    amax=np.array(annualmaxsort)[:,0:n]
    return annualmaxsort
Annualmax = annualmax(Q, df_fut, m*n)

In [22]:
# calculating 50-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/50)
flood50Y5=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood50Y5 = np.array(flood50Y5)

In [23]:
# calculating 100-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/100)
flood100Y5=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood100Y5 = np.array(flood100Y5)

In [24]:
# calculating 500-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/500)
flood500Y5=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood500Y5 = np.array(flood500Y5)

In [25]:
np.save('flood50Y5.npy',flood50Y5)
np.save('flood100Y5.npy',flood100Y5)
np.save('flood500Y5.npy',flood500Y5)